
# The annotated transformer
Cette traduction / adaptation est 

Basée sur une traduction "The annotated transformer" de Austin Huang et al. : http://nlp.seas.harvard.edu/annotated-transformer/ 

Basée sur "The annotated transformer" de Sasha Alexander Rush et al. : https://nlp.seas.harvard.edu/2018/04/03/attention.html

Basée sur "Attention is all you Need" de Hashish Vaswani et al. : https://arxiv.org/abs/1706.03762

https://doi.org/10.48550/arxiv.1706.03762,
doi = {10.48550/ARXIV.1706.03762},
url = {https://arxiv.org/abs/1706.03762},
author = {Vaswani, Ashish and Shazeer, Noam and Parmar, Niki and Uszkoreit, Jakob and Jones, Llion and Gomez, Aidan N. and Kaiser, Lukasz and Polosukhin, Illia},
keywords = {Computation and Language (cs.CL), Machine Learning (cs.LG), FOS: Computer and information sciences, FOS: Computer and information sciences},
title = {Attention Is All You Need},
publisher = {arXiv},
year = {2017},
copyright = {arXiv.org perpetual, non-exclusive license}






<center>
  <p><a href="https://arxiv.org/abs/1706.03762">Attention is All You Need
  </a></p>
  <p>v2022: Austin Huang, Suraj Subramanian, Jonathan Sum, Khalid Almubarak,
    and Stella Biderman.</p>
  <p>[Original](https://nlp.seas.harvard.edu/2018/04/03/attention.html):
   <a href="http://rush-nlp.com/">Sasha Rush</a></p>
   <p>code available <a href="https://github.com/harvardnlp/annotated-transformer">here</a></p>
</center>

---


# Abstract



## But

L'objectif de réduire les calculs séquentiels forme aussi les bases du Neural GPU étendu, ByteNet et ConvS25 et tout ce qui utilise les réseaux de neurones convolutionnels sont des blocs de constructions basiques, calculant des représentations cachées en parrallèle pour toutes les positions d'entrées et de sorties. 

Dans ces modèles le nombre d'opérations requises pour relier les signaux de deux entrées arbitraires augmente avec la distance entre les entrées 

- de façon linéaire pour ConvS2S et 

- de façon logarithmique avec ByteNet. 

Ceci rend plus difficile l'apprentissage des dépendances entre inputs distants bien qu'au prix de résolution effective réduite en raison de la moyenne pondérée par l'attention positions, un effet que nous contrecarrons avec une attention multi-tête.

## Auto-attention

Self-attention, sometimes called intra-attention is an attention
mechanism relating different positions of a single sequence in order
to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading
comprehension, abstractive summarization, textual entailment and
learning task-independent sentence representations. End-to-end
memory networks are based on a recurrent attention mechanism instead
of sequencealigned recurrence and have been shown to perform well on
simple-language question answering and language modeling tasks.

To the best of our knowledge, however, the Transformer is the first
transduction model relying entirely on self-attention to compute
representations of its input and output without using sequence
aligned RNNs or convolution.

# Configuration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/annotated_transformer_FR-EN/

!pip install -r requirements.txt
!pip install -q tfds-nightly tensorflow matplotlib
!pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
from torchdata.datapipes.iter import IterableWrapper, Mapper
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from IPython.display import Image, display


# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True


# Load spacy tokenizer models, download them if they haven't been
# downloaded already
def load_tokenizers():

    try:
        spacy_fr = spacy.load("fr_core_news_sm")
    except IOError:
        os.system("python -m spacy download fr_core_news_sm")
        spacy_fr = spacy.load("fr_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_fr, spacy_en


# Some convenience helper functions used throughout the notebook
def is_interactive_notebook():
    return __name__ == "__main__"

def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)

def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

# Modèle

## Architecture

La plupart des modèle neuronaux "transduction sequence" ont une structure <b>encoder-decoder</b> [(cite)](https://arxiv.org/abs/1409.0473).

Ici l'encodeur implémente une séquence d'input de symboles de représentation $(x_1, ..., x_n)$ vers une séquence de symboles de représentation $\mathbf{z} = (z_1, ..., z_n)$ où $\mathbf{z \in R}$

Soit $\mathbf{z \in R}$, le décodeur génère une séquence de sortie $(y_1,...,y_m)$ avec un élément à la fois, à chaque étape le modèle est "auto-régressif" [(cite)](https://arxiv.org/abs/1308.0850) consomant le symbole précédement généré comme entrée additionnelle pour générer le suivant.

Le transformers suit cette structure générale en utilisant une pile auto-attention point par points, avec des couches totalement connectée pour l'encodeur et le décodeur, comme c'est montré dans les moitiés gauche et droite de la figure 1.

<!-- <img src="images/aiayn.png" width="70%"/> -->

In [ ]:
# display(Image('images/ModalNet-21.png'))

In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

### Encodeur

L'encodeur est composée d'une pile de $N=6$ couches iddentiques.

Nous employons une couche résiduelle [(cite)](https://arxiv.org/abs/1512.03385) autour de chacune des deux sous couches, suivie par la couche de normalisation [(cite)](https://arxiv.org/abs/1607.06450).

In [ ]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

Donc, la sortie de chaque sous couche est $\mathrm{LayerNorm}(x +
\mathrm{Sublayer}(x))$, où $\mathrm{Sublayer}(x)$ est la fonction implémentée par la sous-couche elle même.

Nous appliquons une dropout [(cite)](http://jmlr.org/papers/v15/srivastava14a.html) à la sortie de chaque sous-couches avant c'est ajouté à la sortie de la sous-couche et normalité.

Pour faciliter ces connections résiduelles, chaque sous-couche du modèle, ainsi que les couches d'embedding produisent une sortie de dimension $d_{\text{model}}=512$.

Chaque couche à deux sous couches :

- La première est un méchanisme d'attention multi-tête.
- La seconde est un simple réseau d'alimentation entièrement connecté.

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"

    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

### Decoder

Le décoder est aussi composée d'une pile de $N=6$ couches iddentiques.

En plus des deux sous-couches dans chaque couche de l'encoder, le décodeur insère une 3° sous-couche, qui effectue une attention multi tête au dessus de la sortie de la pile de l'encodeur.



In [ ]:
# display(Image('images/ModalNet-19.png'))

De façon similaireà l'encodeur nous utilisons une connection résiduelle autour de chaque sous couche, suivie par une couche de normalisation.

In [ ]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."

    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

Nous modifions également la sous-couche d'auto-attention dans la pile du décodeur pour prévenir les indices de positions d'être sous-évaluées.

#### Masque
Ce masque, combiné au fait que les plongements de sortie sont compensés par une position, assure que les prédictions pour la position $i$ ne peuvent dépendre que des sorties connues aux positions inférieures à $i$.

Ci-dessous le masque d'attention montre que la position de chaque mots d'objectifs (ligne) est aloué comme une colonne. Les mots sont bloqués pour recevoir des mots futurs pendant l'entrainement.

In [ ]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    # Returns the upper triangular part of a matrix (2-D tensor) or batch of matrices input, the other elements of the result tensor out are set to 0
    # https://pytorch.org/docs/stable/generated/torch.triu.html
        
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

def example_mask():
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Subsequent Mask": subsequent_mask(20)[0][x, y].flatten(),
                    "Window": y,
                    "Masking": x,
                }
            )
            for y in range(20)
            for x in range(20)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect()
        .properties(height=250, width=250)
        .encode(
            alt.X("Window:O"),
            alt.Y("Masking:O"),
            alt.Color("Subsequent Mask:Q", scale=alt.Scale(scheme="viridis")),
        )
        .interactive()
    )

show_example(example_mask)

alt.Chart(...)

## Attention

Soit une fonction d'attention où $Q, K, V \in R^2$ implémentant une requête $Q$ et un ensemble de (clé $K$, valeur $V$) vers une sortie calculée comme une somme pondérée des valeurs où 



### Attention additive et attention multiplicative

Nous appelons <i>Scaled Dot-Product Attention</i>, une attention telle que $\forall$ valeur $V$, le poids assigné est calculé par une <b>fonction de compatibilité</b> de la requête $Q$ avec la clé correspondante $K$.

L'entrée consiste en requêtes et clés de dimensions $d_k$ et valeurs de dimensions $d_v$. 

Nous divisons $Q · V$ par $\sqrt{d_k}$ et appliquons une fonction softmax pour obtenir le poids multiplié par les les valeurs.


$$
   \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$

In [ ]:
# In practice, we compute the matrix of outputs on a set of queries simultaneously, packed together into a matrix Q. 
# The keys and values are also packed together into matrices K and V.

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

# display(Image('images/ModalNet-20.png'))

Les deux fonctions d'attention les plus communément utilisée sont l'attention additive [(cite)](https://arxiv.org/abs/1409.0473) et l'attention multiplicative (dot-product).

L'attention multiplicative est iddentique à notre algo excepté pour le facteur de mise à l'échelle $\frac{1}{\sqrt{d_k}}$.

Une idée de base de la mise à l'échelle est d'éviter les valeurs trop éloignées les unes des autres, d'un autre côté tout s'ajuste autour des valeurs les plus courantes, les autres probabilités devenant moins visibles jusqu'a disparaitre.

L'attention additive calcule la fonction de compatibilité en utilisant un réseau feed-forward avec une simple couche cachée. Alors que les deux ont une complexité similaire d'un point de vue théorique, l'attention dot-product est plus rapide et pus efficiente d'un point de vue espace en pratique puisqu'elle peut être implémentée en utilisant une matrice de multiplication de code autement optimisée.

Tandis que pour les petites valeurs de $d_k$ les deux méchanismes ont des performances similaires, l'attention additive est meilleure que l'attention multiplicative sans mise à échelle pour de plus grande valeurs de $d_k$, les dot products grandissent fort en magnitude poussant la fonction softmax dans des régions où elle a des gradients très petits.

Pour illustrer pourquoi le dot product grandit fort, nous supposons que les éléments de $q$ et $k$ sont des variables indépendantes de moyenne $0$ et de variance $1$. Alors leurs dot products, $q \cdot k = \sum_{i=1}^{d_k} q_ik_i$ ont une moyenne $0$ et une variance $d_k$.

Pour contrecarer cet effet, nous mettons le dot product à échelle avec $\frac{1}{\sqrt{d_k}}$.


### Attention multi-tête

L'attention multi tête permet au modèle de recevoir conjointement de l'information de différent espaces de représentation à plusieurs positions. Avec une seule tête d'attention, la moyenne inhibe cela.

$$
\mathrm{MultiHead}(Q, K, V) =
    \mathrm{Concat}(\mathrm{head_1}, ..., \mathrm{head_h})W^O \\
    \text{où}~\mathrm{head_i} = \mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i)
$$



#### Projections

Les projections sont des matrices de paramètres $W^Q_i \in
\mathbb{R}^{d_{\text{model}} \times d_k}$, $W^K_i \in
\mathbb{R}^{d_{\text{model}} \times d_k}$, $W^V_i \in
\mathbb{R}^{d_{\text{model}} \times d_v}$ and $W^O \in
\mathbb{R}^{hd_v \times d_{\text{model}}}$.

#### Couches

Avec cet ouvrage nous employons $h=8$ couches d'attentions (ou têtes) de manières parrallèles. Pour chaque nous utilisons $d_k=d_v=d_{\text{model}}/h=64$.

#### Réduction de la dimensionalité

Avec le faite de réduire la dimension de chaque tête le coût total compatutionnel est similaire à celui d'un modèle d'attention avec une seule tête avec une dimensionalité complète.

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        # self.linears = copy.deepcopy(nn.Linear(d_model, d_model)) # is not iterable?
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        # 3) "Concat" using a view and apply a final linear.
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x)

#### La mise en application de l'attention dans notre modèle

Les transformers utilisent les attentions multi-têtes de 3 façons différentes :

- Dans les couches d'attention encodeur-décodeur, la requête provient de la couche de décoder précédente et les clés / valeurs enregistrées proviennent de la sortie de l'encoder. Cela permet à chaque positions dans le décoder de rejoindre l'encodeur. Cela permet à chaque positions du decodeur d'atteindre toutes les positions de la séquence d'entrées. Cela imite le sequence-to-sequance de la façon de [(cite)](https://arxiv.org/abs/1609.08144).

- L'encodeur contient une couche d'auto attention. Dans une couche d'auto attention toutes les clés, valeurs, requêtes viennent du même endroit. Dans ce cas la sortie de la couche précédente dans l'encodeur. Chaque positions dans l'encodeur peu atteindre chaque positions dans la couche précédente de l'encodeur.

- De façon similaire, les couches d'auto-attention dans le décodeur permet à chaque position dans le décodeur d'atteindre toutes les potitions dans le décodeur afin d'en inclure la position. Nous voulons prévenir les informations venant de gauche d'arriver dans le décodeur pour préserver la propriété d'<b>auto regressivité</b>. Nous implémentons cela dans l'attention multiplicative mise à échelle en masquant (mise à $-\infty$) toutes les valeurs dans l'entrée de la softmax qui correspond à une connection illégale.

## Réseaux Feed-Forward en fonction de la position

En plus des sous-couches d'attention, chacune des couches dans notre encoder/décodeur contient un réseau feed-forward totalement connecté, ce qui est appliqué à chaque positions séparément et de manière iddentique. Cela consiste à deux transformation linéaires avec une activation ReLU entre les deux.

$$\mathrm{FFN}(x)=\max(0, xW_1 + b_1) W_2 + b_2$$

Alors que les transformations linéaires sont les mêmes au travers de différentes positions, elles utilisent différents paramètres de couches en couches. Une autre façon de décrire cela est une double convolution de taille 1. La dimensionalité des entrées et sorties est $d_{\text{model}}=512$, et la couche interne à une dimensionalité de $d_{ff}=2048$.

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

## Embeddings et Softmax

De façon similaire aux autres modèles de séquences de transduction, nous utilisons des embeddings (encastrements?) pour convertir les tokens d'entrées et de sorties en vecteurs de dimension $d_{\text{model}}$. 

Nous utilisons aussi la transformation linéaire apprise et la fonction softmax pour convertir la sortie du décodeur en prédictions pour la probabilité du token suivant. Dans notre modèle, nous partageons une matrice de même poids entre les deux couches d'embeddings et la transformation linéaire pré-softmax, de façon similaire à [(cite)](https://arxiv.org/abs/1608.05859).

Dans la couche embedded nous multiplions ces poids par $\sqrt{d_{\text{model}}}$.

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

### Encodage positionnel

Puisque notre modèle ne contient ni récurence ni convolutions, afin pour le modèle d'ordonner la séquence, nous devons injecter quelques informations à propos de la position relative ou absolue des tokens dans la séquence. A cette fin, nous ajoutons un "encodage positionnel" à l'embedding d'entrée en bas des piles d'encodage et de décodage. L'encodage positionnel à la même dimension $d_{\text{model}}$ que les embeddings, donc les deux peuvent être sommés.

Il y a beaucoup de choix d'encodage positionnels, appris et fixés [(cite)](https://arxiv.org/pdf/1705.03122.pdf).



Dans cet ouvrage, nous utilisons les fonction sinus et cosinus à différentes fréquences :

$$PE_{(pos,2i)} = \sin(pos / 10000^{2i/d_{\text{model}}})$$

$$PE_{(pos,2i+1)} = \cos(pos / 10000^{2i/d_{\text{model}}})$$

où $pos$ est la position et $i$ est la dimension. Ainsi, chaque dimension de l'encodage positionnel correspond à une sinusoid. a longueur de la vague forme une progression géométrique de $2\pi$ to $10000 \cdot 2\pi$.

Nous choisissons cette fonction sous l'hypothèse que ça permet au modèle de facilement apprendre à atteindre les positions relatives, puisque pour chaque déplacements fixes $k$, $PE_{pos+k}$ peut être représentée comme une fonction linéaire de $PE_{pos}$.

En plus, nous appliquons une dropout à la somme des embeddings et de l'encodage positionnel dans les deux piles (encodeur et décodeur). Pour le modèle de base, nous utilisons un rating de $P_{drop}=0.1$.


In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

Ci-dessus l'encodage positionnel ajoute dans une basée sur une sinusoide. La fréquence et le déplacement de la vague sont différents pour chaque dimension.

In [ ]:
# def example_positional():
#     pe = PositionalEncoding(20, 0)
#     y = pe.forward(torch.zeros(1, 100, 20))

#     data = pd.concat(
#         [
#             pd.DataFrame(
#                 {
#                     "embedding": y[0, :, dim],
#                     "dimension": dim,
#                     "position": list(range(100)),
#                 }
#             )
#             for dim in [4, 5, 6, 7]
#         ]
#     )

#     return (
#         alt.Chart(data)
#         .mark_line()
#         .properties(width=800)
#         .encode(x="position", y="embedding", color="dimension:N")
#         .interactive()
#     )


# show_example(example_positional)

Nous expérimentons également en utilisant un embedding positionnel appris [(cite)](https://arxiv.org/pdf/1705.03122.pdf) à la et constatons que les deux versions produisent des résultats fort proches. Nous choisissons la version sinuzoidale car elle peu permettre au modèle d'extrapoler une plus grande longueur de séquence que celle rencontrée durant l'entrainement.

## Modèle complet
Nous définissons ici une fonction à partir d'hyperparamètres vers un modèle complet.

In [ ]:
def make_model(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

# Entrainement
Nous stoppons pour une rapide interlude pour introduire quelque uns des outils dont nous avons besoins pour entrainer un modèle encodeur-décodeur standard. D'abord nous définitssons un objet batch qui gère les phrases sources et objectives pour l'entrainement aussi bien que qui construit les masques.

## Batchs

Nous gérons des batches de données auquel nous ajoutons un masque pour masquer le padding et les mots qui suivent.

In [ ]:
class Batch:
    """Object for holding a batch of data with mask during training."""

    def __init__(self, src, tgt=None, pad=2):  # 2 = <blank>
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            self.tgt = tgt[:, :-1]
            self.tgt_y = tgt[:, 1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y != pad).data.sum()

    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask

## Training Loop

Créons un entrainement générique et une fonction de coût pour garder une trace de la loss. Nous passons par une fonction de loss générale qui gère également les paramètres de mise à jour.

In [ ]:
class TrainState:
    """Track number of steps, examples, and tokens processed"""

    step: int = 0  # Steps in the current epoch
    accum_step: int = 0  # Number of gradient accumulation steps
    samples: int = 0  # total # of examples used
    tokens: int = 0  # total # of tokens processed
  
def run_epoch(
  data_iter,
  model,
  loss_compute,
  optimizer,
  scheduler,
  mode="train",
  accum_iter=1,
  train_state=TrainState(),
  ):
    """Train a single epoch"""
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
        # loss_node = loss_node / accum_iter
        if mode == "train" or mode == "train+log":
            loss_node.backward()
            train_state.step += 1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
                train_state.accum_step += 1
            scheduler.step()

        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            )
            start = time.time()
            tokens = 0
        del loss
        del loss_node
    return total_loss / total_tokens, train_state

## Optimiseurs

Nous avons utilisé l'optimiseur  [(Adam)](https://arxiv.org/abs/1412.6980) avec 

- $\beta_1=0.9$, 

- $\beta_2=0.98$ et 

- $\epsilon=10^{-9}$. 

Nous varions le taux d'apprentissage au travers de la course d'entrainement avec la formule :

$$
lrate = d_{\text{model}}^{-0.5} \cdot
  \min({step\_num}^{-0.5},
    {step\_num} \cdot {warmup\_steps}^{-1.5})
$$


Ceci correspond à augmenter le taux d'apprentissage de façon linéaire de la première $warmup\_steps$ étape d'entrainement, et la diminuer ensuite ptoportionellement à l'inverse de la racine carré du numéro d'étape. 

Nous utilisons $warmup\_steps=4000$.

In [ ]:
# Notons que cette partie est très importante. Nous devons faire l'entrainement avec cette configuration du modèle.
def rate(step, model_size, factor, warmup):
    """
    we have to default the step to 1 for LambdaLR function
    to avoid zero raising to negative power.
    """
    if step == 0:
        step = 1
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

def example_learning_schedule():
    opts = [
        [512, 1, 4000],  # example 1
        [512, 1, 8000],  # example 2
        [256, 1, 4000],  # example 3
    ]

    dummy_model = torch.nn.Linear(1, 1)
    learning_rates = []

    # we have 3 examples in opts list.
    for idx, example in enumerate(opts):
        # run 20000 epoch for each example
        optimizer = torch.optim.Adam(
            dummy_model.parameters(), lr=1, betas=(0.9, 0.98), eps=1e-9
        )
        lr_scheduler = LambdaLR(
            optimizer=optimizer, lr_lambda=lambda step: rate(step, *example)
        )
        tmp = []
        # take 20K dummy training steps, save the learning rate at each step
        for step in range(20000):
            tmp.append(optimizer.param_groups[0]["lr"])
            optimizer.step()
            lr_scheduler.step()
        learning_rates.append(tmp)

    learning_rates = torch.tensor(learning_rates)

    # Enable altair to handle more than 5000 rows
    alt.data_transformers.disable_max_rows()

    opts_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Learning Rate": learning_rates[warmup_idx, :],
                    "model_size:warmup": ["512:4000", "512:8000", "256:4000"][
                        warmup_idx
                    ],
                    "step": range(20000),
                }
            )
            for warmup_idx in [0, 1, 2]
        ]
    )

    return (
        alt.Chart(opts_data)
        .mark_line()
        .properties(width=600)
        .encode(x="step", y="Learning Rate", color="model_size:warmup:N")
        .interactive()
    )

# Example des courbes de ce modèle pour différentes tailles de modèles et paramètres d'optimizations.
example_learning_schedule()

Output hidden; open in https://colab.research.google.com to view.

## Régularization

Durant l'entrainement, nous employons le lissage de libellés (label smoothing) de valeurs $\epsilon_{ls}=0.1$ [(cite)](https://arxiv.org/abs/1512.00567).

Nous implémentons le lissage de libéllé utilisant la KL div loss. A la place d'utiliser une distribution objective one-hot, nous créons une distribution qui à une `confidence` (confiance) du mot correct et du reste du `smoothing` (lissage) de mass distribulé au travers du vocabulaire.

In [ ]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())

# Voir comment la masse est distribuée aux mots basés sur la confiance.

def example_label_smoothing():
    crit = LabelSmoothing(5, 0, 0.4)
    predict = torch.FloatTensor(
        [
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
        ]
    )
    crit(x=predict.log(), target=torch.LongTensor([2, 1, 0, 3, 3]))
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "target distribution": crit.true_dist[x, y].flatten(),
                    "columns": y,
                    "rows": x,
                }
            )
            for y in range(5)
            for x in range(5)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect(color="Blue", opacity=1)
        .properties(height=200, width=200)
        .encode(
            alt.X("columns:O", title=None),
            alt.Y("rows:O", title=None),
            alt.Color(
                "target distribution:Q", scale=alt.Scale(scheme="viridis")
            ),
        )
        .interactive()
    )

show_example(example_label_smoothing)

alt.Chart(...)

- Améliore la précision (accuracy) et le BLEU score.

- Le lissage de libéllé commence actuellement à pénaliser le modèle s'il devient très confiant à propos d'un choix donné.

---

NB : Label smoothing

    Turns “hard” class label assignments to “soft” label assignments.
    Operates directly on the labels themselves.
    Is dead simple to implement.
    Can lead to a model that generalizes better.

source : https://pyimagesearch.com/2019/12/30/label-smoothing-with-keras-tensorflow-and-deep-learning/

---

In [ ]:
def loss(x, crit):
    d = x + 3 * 1
    predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d]])
    return crit(predict.log(), torch.LongTensor([1])).data


def penalization_visualization():
    crit = LabelSmoothing(5, 0, 0.1)
    loss_data = pd.DataFrame(
        {
            "Loss": [loss(x, crit) for x in range(1, 100)],
            "Steps": list(range(99)),
        }
    ).astype("float")

    return (
        alt.Chart(loss_data)
        .mark_line()
        .properties(width=350)
        .encode(
            x="Steps",
            y="Loss",
        )
        .interactive()
    )

show_example(penalization_visualization)

alt.Chart(...)

# Un premier exemple

Nous pouvons commencer par entrainer une simple "copy-task". Soit un ensemble aléatoire de symboles d'entrées pris sur un petit vocabulaire, l'objective est de générer en retour les mêmes symboles.


## Clacul de la fonction de coût

In [ ]:
class SimpleLossCompute:
    "A simple loss compute and train function."

    def __init__(self, generator, criterion):
        self.generator = generator
        self.criterion = criterion

    def __call__(self, x, y, norm):
        x = self.generator(x)
        sloss = (
            self.criterion(
                x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)
            )
            / norm
        )
        return sloss.data * norm, sloss

## Décodage (Greedy Decoding)

Choix du greedy pour le code suivant effectué par simplifité.

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys

# Train the simple copy task.
def example_simple_model():
    V = 11
    criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
    model = make_model(V, V, N=2)

    optimizer = torch.optim.Adam(
        model.parameters(), lr=0.5, betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, model_size=model.src_embed[0].d_model, factor=1.0, warmup=400
        ),
    )

    batch_size = 80
    for epoch in range(20):
        model.train()
        run_epoch(
            data_gen(V, batch_size, 20),
            model,
            SimpleLossCompute(model.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train",
        )
        model.eval()
        run_epoch(
            data_gen(V, batch_size, 5),
            model,
            SimpleLossCompute(model.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )[0]

    model.eval()
    src = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
    max_len = src.shape[1]
    src_mask = torch.ones(1, 1, max_len)
    print(greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0))


# execute_example(example_simple_model)  # 10 minutes and produce :

# Epoch Step:      1 | Accumulation Step:   2 | Loss:   3.42 | Tokens / Sec:   571.5 | Learning Rate: 5.5e-06
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   2.10 | Tokens / Sec:   582.6 | Learning Rate: 6.1e-05
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.75 | Tokens / Sec:   618.7 | Learning Rate: 1.2e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.47 | Tokens / Sec:   614.8 | Learning Rate: 1.7e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.12 | Tokens / Sec:   607.5 | Learning Rate: 2.3e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.68 | Tokens / Sec:   609.8 | Learning Rate: 2.8e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.38 | Tokens / Sec:   619.2 | Learning Rate: 3.4e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.31 | Tokens / Sec:   596.9 | Learning Rate: 3.9e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.11 | Tokens / Sec:   479.2 | Learning Rate: 4.5e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.13 | Tokens / Sec:   540.8 | Learning Rate: 5.0e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.21 | Tokens / Sec:   618.5 | Learning Rate: 5.6e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.11 | Tokens / Sec:   619.9 | Learning Rate: 6.1e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.12 | Tokens / Sec:   560.5 | Learning Rate: 6.7e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.17 | Tokens / Sec:   612.3 | Learning Rate: 7.2e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.11 | Tokens / Sec:   535.3 | Learning Rate: 7.8e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.11 | Tokens / Sec:   609.4 | Learning Rate: 8.3e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.16 | Tokens / Sec:   463.4 | Learning Rate: 8.9e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.16 | Tokens / Sec:   609.9 | Learning Rate: 9.4e-04
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.35 | Tokens / Sec:   608.2 | Learning Rate: 1.0e-03
# Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.07 | Tokens / Sec:   604.9 | Learning Rate: 1.1e-03
# tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

## Chargement des données

Nous chargeons le dataset utilisant torchtext et spacy pour la tokenization.



In [ ]:
import tensorflow_datasets as tfds
import numpy as np
import tensorflow as tf

def retrieveMtnt() :
  ds = tfds.load('mtnt', split='train', shuffle_files=True)
  assert isinstance(ds, tf.data.Dataset)
  print(ds)

  builder = tfds.builder('mtnt')
  builder.download_and_prepare()
  ds = builder.as_dataset(split='train', shuffle_files=True)

  train = ds
  map_train = Mapper(train, lambda x: x)
  val = []
  for i,j in enumerate(list(map_train)):
    str_fr = str(np.array(j.get("dst")))[2:-1]
    str_en = str(np.array(j.get("src")))[2:-1]
    res = str_fr, str_en
    val.append(res)
  return val

def tokenize(text, tokenizer):
    return [tok.text for tok in tokenizer.tokenizer(text)]

def yield_tokens(data_iter, tokenizer, index):
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])

def build_vocabulary(spacy_fr, spacy_en):
    def tokenize_fr(text):
        return tokenize(text, spacy_fr)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    print("Building French Vocabulary ...")
    # train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    
    vocab_src = build_vocab_from_iterator(
        yield_tokens(retrieveMtnt(), tokenize_fr, index=0),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    print("Building English Vocabulary ...")
    # train, val, test = datasets.Multi30k(language_pair=("de", "en"))

    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(retrieveMtnt(), tokenize_en, index=1),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"])

    return vocab_src, vocab_tgt


def load_vocab(spacy_fr, spacy_en):
    if not exists("vocab.pt"):
        vocab_src, vocab_tgt = build_vocabulary(spacy_fr, spacy_en)
        torch.save((vocab_src, vocab_tgt), "vocab.pt")
    else:
        vocab_src, vocab_tgt = torch.load("vocab.pt")
    print("Loading finished!")
    print("source vocabulary size : ", len(vocab_src))
    print("target vocabulary size : ", len(vocab_tgt))
    return vocab_src, vocab_tgt


if is_interactive_notebook():
    # global variables used later in the script
    spacy_fr, spacy_en = show_example(load_tokenizers)
    vocab_src, vocab_tgt = show_example(load_vocab, args=[spacy_fr, spacy_en])

Building French Vocabulary ...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Generating test examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/mtnt/en-fr/1.0.0.incompleteD4IQJR/mtnt-test.tfrecord*...:   0%|          |…

Generating valid examples...:   0%|          | 0/811 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/mtnt/en-fr/1.0.0.incompleteD4IQJR/mtnt-valid.tfrecord*...:   0%|          …

Dataset mtnt downloaded and prepared to /root/tensorflow_datasets/mtnt/en-fr/1.0.0. Subsequent calls will reuse this data.
<PrefetchDataset element_spec={'dst': TensorSpec(shape=(), dtype=tf.string, name=None), 'src': TensorSpec(shape=(), dtype=tf.string, name=None)}>
Building English Vocabulary ...
<PrefetchDataset element_spec={'dst': TensorSpec(shape=(), dtype=tf.string, name=None), 'src': TensorSpec(shape=(), dtype=tf.string, name=None)}>
Loading finished!
source vocabulary size :  32418
target vocabulary size :  28272


Faire des sous ensemble (batching) compte énormément pour la vitesse. Nous voulons des sous enemble de même tailles avec un padding minimal. Hackons par rapport au batching par défault de torchtext. Cette modification de code résouds la problématique de leurs version de base pour nous assurer que nous cherchons assez de phrases pour trouver des lots sérrés (tight batches).


## Iterateurs

In [ ]:
def collate_batch(
    batch,
    src_pipeline,
    tgt_pipeline,
    src_vocab,
    tgt_vocab,
    device,
    max_padding=128,
    pad_id=2,
):
    bs_id = torch.tensor([0], device=device)  # <s> token id
    eos_id = torch.tensor([1], device=device)  # </s> token id
    src_list, tgt_list = [], []
    for (_src, _tgt) in batch:
        processed_src = torch.cat(
            [
                bs_id,
                torch.tensor(
                    src_vocab(src_pipeline(_src)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    tgt_vocab(tgt_pipeline(_tgt)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        src_list.append(
            # warning - overwrites values for negative values of padding - len
            pad(
                processed_src,
                (
                    0,
                    max_padding - len(processed_src),
                ),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_padding - len(processed_tgt)),
                value=pad_id,
            )
        )

    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)
    return (src, tgt)

In [ ]:
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

def create_dataloaders(
    device,
    vocab_src,
    vocab_tgt,
    spacy_fr,
    spacy_en,
    batch_size=12000,
    max_padding=128,
    is_distributed=True,
):
    # def create_dataloaders(batch_size=12000):
    def tokenize_fr(text):
        return tokenize(text, spacy_fr)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_fr,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
        )

    # train_iter, valid_iter, test_iter = datasets.Multi30k(
    #     language_pair=("fr", "en")
    # )

    full = retrieveMtnt()
    train_iter, tmp_iter = split_list(full)
    
    test_iter, valid_iter = split_list(tmp_iter)

    train_iter_map = to_map_style_dataset(
        train_iter
    )  # DistributedSampler needs a dataset len()
    train_sampler = (
        DistributedSampler(train_iter_map) if is_distributed else None
    )
    valid_iter_map = to_map_style_dataset(valid_iter)
    valid_sampler = (
        DistributedSampler(valid_iter_map) if is_distributed else None
    )

    train_dataloader = DataLoader(
        train_iter_map,
        batch_size=batch_size,
        shuffle=(train_sampler is None),
        sampler=train_sampler,
        collate_fn=collate_fn,
    )
    valid_dataloader = DataLoader(
        valid_iter_map,
        batch_size=batch_size,
        shuffle=(valid_sampler is None),
        sampler=valid_sampler,
        collate_fn=collate_fn,
    )
    return train_dataloader, valid_dataloader

## L'entrainement du sytème

In [ ]:
def train_worker(
    gpu,
    ngpus_per_node,
    vocab_src,
    vocab_tgt,
    spacy_fr,
    spacy_en,
    config,
    is_distributed=False,
):
    print(f"Train worker process using GPU: {gpu} for training", flush=True)
    torch.cuda.set_device(gpu)

    pad_idx = vocab_tgt["<blank>"]
    d_model = 512
    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.cuda(gpu)
    module = model
    is_main_process = True
    if is_distributed:
        dist.init_process_group(
            "nccl", init_method="env://", rank=gpu, world_size=ngpus_per_node
        )
        model = DDP(model, device_ids=[gpu])
        module = model.module
        is_main_process = gpu == 0

    criterion = LabelSmoothing(
        size=len(vocab_tgt), padding_idx=pad_idx, smoothing=0.1
    )
    criterion.cuda(gpu)

    train_dataloader, valid_dataloader = create_dataloaders(
        gpu,
        vocab_src,
        vocab_tgt,
        spacy_fr,
        spacy_en,
        batch_size=config["batch_size"] // ngpus_per_node,
        max_padding=config["max_padding"],
        is_distributed=is_distributed,
    )

    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["base_lr"], betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, d_model, factor=1, warmup=config["warmup"]
        ),
    )
    train_state = TrainState()

    for epoch in range(config["num_epochs"]):
        if is_distributed:
            train_dataloader.sampler.set_epoch(epoch)
            valid_dataloader.sampler.set_epoch(epoch)

        model.train()
        print(f"[GPU{gpu}] Epoch {epoch} Training ====", flush=True)
        _, train_state = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train+log",
            accum_iter=config["accum_iter"],
            train_state=train_state,
        )

        GPUtil.showUtilization()
        if is_main_process:
            file_path = "%s%.2d.pt" % (config["file_prefix"], epoch)
            torch.save(module.state_dict(), file_path)
        torch.cuda.empty_cache()

        print(f"[GPU{gpu}] Epoch {epoch} Validation ====", flush=True)
        model.eval()
        sloss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in valid_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )
        print(sloss)
        torch.cuda.empty_cache()

    if is_main_process:
        file_path = "%sfinal.pt" % config["file_prefix"]
        torch.save(module.state_dict(), file_path)

In [ ]:
def train_worker(
    gpu,
    ngpus_per_node,
    vocab_src,
    vocab_tgt,
    spacy_fr,
    spacy_en,
    config,
    is_distributed=False,
):
    print(f"Train worker process using GPU: {gpu} for training", flush=True)
    torch.cuda.set_device(gpu)

    pad_idx = vocab_tgt["<blank>"]
    d_model = 512
    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.cuda(gpu)
    module = model
    is_main_process = True
    if is_distributed:
        dist.init_process_group(
            "nccl", init_method="env://", rank=gpu, world_size=ngpus_per_node
        )
        model = DDP(model, device_ids=[gpu])
        module = model.module
        is_main_process = gpu == 0

    criterion = LabelSmoothing(
        size=len(vocab_tgt), padding_idx=pad_idx, smoothing=0.1
    )
    criterion.cuda(gpu)

    train_dataloader, valid_dataloader = create_dataloaders(
        gpu,
        vocab_src,
        vocab_tgt,
        spacy_fr,
        spacy_en,
        batch_size=config["batch_size"] // ngpus_per_node,
        max_padding=config["max_padding"],
        is_distributed=is_distributed,
    )

    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["base_lr"], betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, d_model, factor=1, warmup=config["warmup"]
        ),
    )
    train_state = TrainState()

    for epoch in range(config["num_epochs"]):
        if is_distributed:
            train_dataloader.sampler.set_epoch(epoch)
            valid_dataloader.sampler.set_epoch(epoch)

        model.train()
        print(f"[GPU{gpu}] Epoch {epoch} Training ====", flush=True)
        _, train_state = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train+log",
            accum_iter=config["accum_iter"],
            train_state=train_state,
        )

        GPUtil.showUtilization()
        if is_main_process:
            file_path = "%s%.2d.pt" % (config["file_prefix"], epoch)
            torch.save(module.state_dict(), file_path)
        torch.cuda.empty_cache()

        print(f"[GPU{gpu}] Epoch {epoch} Validation ====", flush=True)
        model.eval()
        sloss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in valid_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )
        print(sloss)
        torch.cuda.empty_cache()

    if is_main_process:
        file_path = "%sfinal.pt" % config["file_prefix"]
        torch.save(module.state_dict(), file_path)


In [ ]:

def train_distributed_model(vocab_src, vocab_tgt, spacy_fr, spacy_en, config):
    
    ngpus = torch.cuda.device_count()
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12356"
    print(f"Number of GPUs detected: {ngpus}")
    print("Spawning training processes ...")
    mp.spawn(
        train_worker,
        nprocs=ngpus,
        args=(ngpus, vocab_src, vocab_tgt, spacy_fr, spacy_en, config, True),
    )


def train_model(vocab_src, vocab_tgt, spacy_fr, spacy_en, config):
    if config["distributed"]:
        train_distributed_model(
            vocab_src, vocab_tgt, spacy_fr, spacy_en, config
        )
    else:
        train_worker(
            0, 1, vocab_src, vocab_tgt, spacy_fr, spacy_en, config, False
        )


def load_trained_model():
    config = {
        "batch_size": 32,
        "distributed": False,
        "num_epochs": 8,
        "accum_iter": 10,
        "base_lr": 1.0,
        "max_padding": 72,
        "warmup": 3000,
        "file_prefix": "multi30k_model_",
    }
    model_path = "multi30k_model_final.pt"
    if not exists(model_path):
        train_model(vocab_src, vocab_tgt, spacy_fr, spacy_en, config)

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(torch.load("multi30k_model_final.pt"))
    return model


if is_interactive_notebook():
    model = load_trained_model()

Une fois entrainée nous pouvons décoder le modèle pour produire un enemmble de traductions. Ici nous traduisons simplement la première phrase dans l'ensemble de validation. Ce dataset est parfaitement petit donc les traductions avec greedy search (recherche gourmande?) sont raisonnablement précises.

Ceci couvre en grande partie le modèle transformers en soi. Il existe 4 aspects que nous n'avons pas couvert explicitement. Nous avons aussi ces 4 features implémentées dans [OpenNMT-py](https://github.com/opennmt/opennmt-py).

# Composants supplémentaires: BPE, Search, Averaging

1. BPE/ Word-piece : Nous pouvons utiliser une librairy pour pré-processer les données dans une unité de sous-mots (subword units). Voir l'implémentation de Rico Sennrich's [subword-nmt](https://github.com/rsennrich/subword-nmt) implementation. Ces modèles transforment les données d'entrainement pour ressembler à quelques chose du style `▁Die ▁Protokoll datei ▁kann ▁ heimlich ▁per ▁E - Mail ▁oder ▁FTP ▁an ▁einen ▁bestimmte n ▁Empfänger ▁gesendet ▁werden`

2. Beam Search: Un peu compliqué pour couvrir ici, voir [OpenNMT-py](https://github.com/OpenNMT/OpenNMT-py/) pour une implémentation pytorch.

3. Les embeddings partagées : En utilisant BPE avec un vocabulaire partagé nous pouvons partagés les mêmes poids de vecteurs entre source (src), cible (tgt) et générateur. Voir [(ceci)](https://arxiv.org/abs/1608.05859) pour plus de détails. Le code pour ajouter ça au modèle est disponible ci-dessous.

4. Moyenne de modèles : Le papier fait la moyenne des derniers $k$ checkpoints pour créer un effet d'ensemble. Nous pouvons faire cela à partir du moment où nous avons un tas de modèles (bunch of models). Code ci-dessous.

In [ ]:
# shared embeddings
if False:
    model.src_embed[0].lut.weight = model.tgt_embeddings[0].lut.weight
    model.generator.lut.weight = model.tgt_embed[0].lut.weight

# model averaging
def average(model, models):
    "Average models into model"
    for ps in zip(*[m.params() for m in [model] + models]):
        ps[0].copy_(torch.sum(*ps[1:]) / len(ps[1:]))

# Resultats

In [ ]:
def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt,
    n_examples=15,
    pad_idx=2,
    eos_string="</s>",
):
    results = [()] * n_examples
    for idx in range(n_examples):
        print("\nExample %d ========\n" % idx)
        b = next(iter(valid_dataloader))
        rb = Batch(b[0], b[1], pad_idx)
        greedy_decode(model, rb.src, rb.src_mask, 64, 0)[0]

        src_tokens = [
            vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx
        ]
        tgt_tokens = [
            vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx
        ]

        print(
            "Source Text (Input)        : "
            + " ".join(src_tokens).replace("\n", "")
        )
        print(
            "Target Text (Ground Truth) : "
            + " ".join(tgt_tokens).replace("\n", "")
        )
        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            " ".join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        print("Model Output               : " + model_txt.replace("\n", ""))
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    return results


def run_model_example(n_examples=5):
    global vocab_src, vocab_tgt, spacy_fr, spacy_en

    print("Preparing Data ...")
    _, valid_dataloader = create_dataloaders(
        torch.device("cpu"),
        vocab_src,
        vocab_tgt,
        spacy_fr,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )

    print("Loading Trained Model ...")

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(
        torch.load("baby_translator_model_final.pt", map_location=torch.device("cpu"))
    )

    print("Checking Model Outputs:")
    example_data = check_outputs(
        valid_dataloader, model, vocab_src, vocab_tgt, n_examples=n_examples
    )
    return model, example_data


execute_example(run_model_example)

Preparing Data ...
<PrefetchDataset element_spec={'dst': TensorSpec(shape=(), dtype=tf.string, name=None), 'src': TensorSpec(shape=(), dtype=tf.string, name=None)}>
Loading Trained Model ...
Checking Model Outputs:

Example 0 ========

Source Text (Input)        : <s> [ mw2 ] Que cela ait \xc3\xa9t\xc3\xa9 un simple battage m\xc3\xa9diatique ou non . </s>
Target Text (Ground Truth) : <s> [ mw2 ] Whether this has all been a hype job or not . </s>
Model Output               : <s> [ New ] This is a hard one of one or not . </s>

Example 1 ========

Source Text (Input)        : <s> ASP - excellente id\xc3\xa9e sauf pour les armes multi-classes . Lorsque l' on fait l' objet resp\xc3\xa9c . ; fais en sorte que ce soit un objet * * TOUT * * resp\xc3\xa9c . Je sais qu' il existe des difficult\xc3\xa9s techniques avec diff\xc3\xa9rentes formes mais le meilleur objet resp\xc3\xa9c . te <unk> m\xc3\xaame ta <unk> en se basant sur ton historique d' achat . Si l' entreprise dit non , envisage quand

## Visualization de l'attention

Même avec un decoder greedy la traduction donne bien. Nous pouvons visualizer ça plus loin pour voir ce qui se passe sur chaque couche de l'attention.

In [ ]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    "convert a dense matrix to a data frame with row and column indices"
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s"
                % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s"
                % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        # if float(m[r,c]) != 0 and r < max_row and c < max_col],
        columns=["row", "column", "value", "row_token", "col_token"],
    )


def attn_map(attn, layer, head, row_tokens, col_tokens, max_dim=30):
    df = mtx2df(
        attn[0, head].data,
        max_dim,
        max_dim,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        .properties(height=400, width=400)
        .interactive()
    )

In [ ]:
def get_encoder(model, layer):
    return model.encoder.layers[layer].self_attn.attn


def get_decoder_self(model, layer):
    return model.decoder.layers[layer].self_attn.attn


def get_decoder_src(model, layer):
    return model.decoder.layers[layer].src_attn.attn


def visualize_layer(model, layer, getter_fn, ntokens, row_tokens, col_tokens):
    # ntokens = last_example[0].ntokens
    attn = getter_fn(model, layer)
    n_heads = attn.shape[1]
    charts = [
        attn_map(
            attn,
            0,
            h,
            row_tokens=row_tokens,
            col_tokens=col_tokens,
            max_dim=ntokens,
        )
        for h in range(n_heads)
    ]
    assert n_heads == 8
    return alt.vconcat(
        charts[0]
        # | charts[1]
        | charts[2]
        # | charts[3]
        | charts[4]
        # | charts[5]
        | charts[6]
        # | charts[7]
        # layer + 1 due to 0-indexing
    ).properties(title="Layer %d" % (layer + 1))

## Encoder Self Attention

In [ ]:
def viz_encoder_self():
    model, example_data = run_model_example(n_examples=1)
    example = example_data[
        len(example_data) - 1
    ]  # batch object for the final example

    layer_viz = [
        visualize_layer(
            model, layer, get_encoder, len(example[1]), example[1], example[1]
        )
        for layer in range(6)
    ]
    return alt.hconcat(
        layer_viz[0]
        # & layer_viz[1]
        & layer_viz[2]
        # & layer_viz[3]
        & layer_viz[4]
        # & layer_viz[5]
    )


show_example(viz_encoder_self)

Output hidden; open in https://colab.research.google.com to view.

## Decoder Self Attention

In [ ]:
def viz_decoder_self():
    model, example_data = run_model_example(n_examples=1)
    example = example_data[len(example_data) - 1]

    layer_viz = [
        visualize_layer(
            model,
            layer,
            get_decoder_self,
            len(example[1]),
            example[1],
            example[1],
        )
        for layer in range(6)
    ]
    return alt.hconcat(
        layer_viz[0]
        & layer_viz[1]
        & layer_viz[2]
        & layer_viz[3]
        & layer_viz[4]
        & layer_viz[5]
    )


show_example(viz_decoder_self)

Preparing Data ...
<PrefetchDataset element_spec={'dst': TensorSpec(shape=(), dtype=tf.string, name=None), 'src': TensorSpec(shape=(), dtype=tf.string, name=None)}>
Loading Trained Model ...
Checking Model Outputs:

Example 0 ========

Source Text (Input)        : <s> 10 chansons que vous avez entendu et dont vous ne connaissez pas le nom J' <unk> UNE CARTE GT - R PENDANT TOUT CE TEMPS </s>
Target Text (Ground Truth) : <s> 10 Songs You 've Heard and Do n't Know the Name   I HAD A GT - R <unk> THIS WHOLE TIME </s>
Model Output               : <s> 10   You did you know what you don\xe2\x80\x99 t know the name is <unk> - <unk> - 20 - <unk> - 20 - 10 - 20 </s>


alt.HConcatChart(...)

## Decoder Src Attention

In [ ]:
def viz_decoder_src():
    model, example_data = run_model_example(n_examples=1)
    example = example_data[len(example_data) - 1]

    layer_viz = [
        visualize_layer(
            model,
            layer,
            get_decoder_src,
            max(len(example[1]), len(example[2])),
            example[1],
            example[2],
        )
        for layer in range(6)
    ]
    return alt.hconcat(
        layer_viz[0]
        & layer_viz[1]
        & layer_viz[2]
        & layer_viz[3]
        & layer_viz[4]
        & layer_viz[5]
    )


show_example(viz_decoder_src)

Preparing Data ...
<PrefetchDataset element_spec={'dst': TensorSpec(shape=(), dtype=tf.string, name=None), 'src': TensorSpec(shape=(), dtype=tf.string, name=None)}>
Loading Trained Model ...
Checking Model Outputs:

Example 0 ========

Source Text (Input)        : <s> Fleur de <unk> au Japon . Pourquoi ces personnes <unk> par la culture japonaise sont toujours aussi stupides ? </s>
Target Text (Ground Truth) : <s> Cherry blossom in Japan   Why are <unk> always so dumb ? </s>
Model Output               : <s> TIL <unk>   Why are these people like the war players are still not so much ? </s>


alt.HConcatChart(...)

# Conclusion

En espérant que ce code pourra servir à d'autres ..

Merci à 
- Sasha Rush, Austin Huang, Suraj Subramanian, Jonathan Sum, Khalid Almubarak, Stella Biderman et al.
- Vaswani et al.